In [1]:
import tweepy
from tweepy import OAuthHandler
import csv
import cv2
import numpy as np
import sys
import random
import pandas as pd
import math
import glob
import os
import io
import re
import string
import operator
import itertools
from collections import Counter
from collections import defaultdict 
import json
import nltk
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI
from nltk.tokenize import word_tokenize
import urllib3
feature_list = []

In [2]:
def map_features(tweet):
    words = set(tweet)
    features = {}
    for f in feature_list:
        features[f]= (f in words)
    return features

In [3]:
def remove_extra(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [4]:
def get_features(tweet):
    feature_vec = []
    words = tweet.split()
    for word in words:
        if ((not re.match(r'^https:',word)) and  (not re.match(r'^@',word))):
            word = remove_extra(word)
            word = word.strip('\'"?,.')
            val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", word)
            if (val is None) or (len(word)<=3):
                continue
            else:
                feature_vec.append(word)
    return feature_vec

In [5]:
def process(percentage,f1name,f2name):
    global feature_list
    train_data = []
    test_data = []
    data1 = pd.read_csv(f1name)
    size = (len(data1.index)-1)*percentage
    size/=100
    #size = 10
    label=0
    for i in range(0,size):
        feature_vec = get_features(data1.iloc[i,2])
        train_data.append((feature_vec,label))
        feature_list.extend(feature_vec)
        
    label=1
    data2 = pd.read_csv(f2name)
    for i in range(0,size):
        feature_vec = get_features(data2.iloc[i,2])
        train_data.append((feature_vec,label))
        feature_list.extend(feature_vec)
        
    #print feature_list
    feature_list = list(set(feature_list))
    #print len(feature_list)
    #print test_data
    
    training_set = nltk.classify.util.apply_features(map_features, train_data)
    
    classifier = nltk.NaiveBayesClassifier.train(training_set)
    
    missclassified_tweets=0
    for i in range(size+1,len(data1)):
        if(classifier.classify(map_features(get_features(data1.iloc[i,2])))!=0):
            missclassified_tweets+=1
    
    for i in range(size+1,len(data2)):
        if(classifier.classify(map_features(get_features(data2.iloc[i,2])))!=1):
            missclassified_tweets+=1
    print missclassified_tweets
    print len(data1)+len(data2)
    

In [6]:
if __name__ == '__main__':
    per_train = 30
    while per_train <= 90:
        print 'per_train is %s' % per_train
        f1name = "Clinton_data.csv"
        f2name = "Trump_data.csv"
        process(per_train,f1name,f2name)
        per_train+=10
    

per_train is 30


KeyboardInterrupt: 